<a href="https://colab.research.google.com/github/Srushanth/Intellithing-Mistral-AI/blob/main/notebooks/1.init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
! yes | pip install --quiet llama_index pypdf llama-cpp-python langchain sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.4 MB/s eta 0:00:00


In [12]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

In [13]:
if "google.colab" in str(get_ipython()):
    documents = SimpleDirectoryReader("./").load_data()
else:
    documents = SimpleDirectoryReader("../data/raw/").load_data()

In [14]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

In [18]:
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

total size (MB): 4368.44


4167it [00:23, 176.23it/s]                         
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [22]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
from llama_index import ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [23]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [24]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [25]:
query_engine = index.as_query_engine()
response = query_engine.query("How many chapters are there in total?")

In [28]:
response.response

' There are 20 chapters in total.'

In [29]:
while True:
  query=input()
  response = query_engine.query(query)
  print(response.response)

How many chapters are there in total?


Llama.generate: prefix-match hit


 There are 20 chapters in total.
List the total number of chapters.


Llama.generate: prefix-match hit


 Based on the given context information, it appears that there are 5 chapters in the book "Hands-On Machine Learning with Scikit-Learn, Keras, and Tensorflow - Concepts, Tools, and Techniques to Build Intelligent Systems" by Aurelien Geron. The chapter numbers are [4484, 76, 14, 482, 65], [31, 173, 54, 4484, 76, 14, 482, 65], [31, 17, 45, 2, 42, 98, 5556, 3, 123, 1], [20, 10, 53, 27, 50, 13, 3, 5696, 173, 220], and [17, 64, 47, 91, 3, 125, 24, 11, 5421, 48].
How many pages does the book have?


Llama.generate: prefix-match hit


 Based on the given context information, it appears that there are 5 rows and 5 columns in each of the two tables provided. Therefore, the total number of pages in the book is likely to be 25 (5 x 5). However, without additional information or confirmation from the author, this answer cannot be confirmed as accurate.


KeyboardInterrupt: ignored